# 2022 LOL WORLD CHAMPIONSHIP DA

## DATA LOAD & library load

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py


from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'chart_studio'

In [5]:
PATH = ""

champions = pd.read_csv(PATH + "wc_champions.csv", encoding='utf-8-sig')
players_main = pd.read_csv(PATH + "wc_players_main.csv", encoding='utf-8-sig')
players_pi = pd.read_csv(PATH + "wc_players_play_in.csv", encoding='utf-8-sig')
teams_main = pd.read_csv(PATH + "wc_teams_main.csv", encoding='utf-8-sig')
teams_pi = pd.read_csv(PATH + "wc_teams_play_in.csv", encoding='utf-8-sig')
lol2022season = pd.read_csv(PATH + "2022matchdata.csv" , encoding='utf-8-sig')
lck2022season = pd.read_csv(PATH + "lck2022.csv" , encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_24688\1530908161.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  lol2022season = pd.read_csv(PATH + "2022matchdata.csv" , encoding='utf-8-sig')


### DATA 확인 및 결측값 확인


> 데이터
- lol2022season
    - 2022시즌 모든 lolesports 경기데이터
- cahmpions
    - 게임 내 champion 통계
- players_main
    - main stage player들의 통계
- players_pi
    - playin stage player들의 통계
- teams_main
    - main stage team들의 통계
- players_main
    - palyin stage team들의 통계

---

관심 column 선택 및 정의서

- 팀분석

    - 게임 고유 ID 매치별, 정규시즌경기 다전제 3판2선승 ->경기승리, 플레이오프 5판3선승 경기승리
    - participantid : (1,2,3...10) 선수데이터, 100블루팀 200레드팀 팀데이터
    - side : blue,red team
    - league : 리그명
    - split : 시즌(spring,summer)
    - team : 팀명
    - playoffs : playoff 유무
    - date : 경기날짜(시간 별 분석시 사용)
    - gamelength : 경기시간(강약팀구분)
    - result(승,패) 0 : 패 1 : 승
    - totalgold 총 골드획득
    - towers
    - exp


| Stats | Definitions |
|------|:-------------------------------------|
gameid	| 게임고유ID( 다전제에서 각 매치별)
participantid	| 선수,팀 데이터 구분


#게임ID #팀인지 선수인지 participantid 100 blue 200 red (1,2,3...10 선수개인) #팀  #리그 #시간 #스플릿 #플레이오프 #승패 

In [4]:
lol2022season.info(show_counts = True)

NameError: name 'lol2022season' is not defined

In [ ]:
lol2022season[ (lol2022season['participantid'] == 100) | (lol2022season['participantid']==200 ) ].isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 24872 entries, 10 to 149231
Columns: 123 entries, gameid to opp_deathsat15
dtypes: float64(93), int64(12), object(18)
memory usage: 23.5+ MB


In [3]:
lol2022season.isna().sum()
#participate가 선수지표면 팀단위의 지표는 결측값이지만. 예외있음 ( 측정되지않거나 기입오류 )
#url은 중국공식통계 링크만 있으므로 필요하지 않음 컬럼

NameError: name 'lol2022season' is not defined

---
날짜 데이터 타입 전처리

In [219]:
lol2022season['date'] = pd.to_datetime(lol2022season['date'], format='%Y-%m-%d %H:%M:%S', errors='raise')

----

> #### 추후 해보자 ( drx 선수별 지표,능력치차트 )

In [182]:
#추후 해보자 ( drx 선수별 지표,능력치차트 )

# lck_spring_po_df = lck_spring_po[ ['gameid','playername','participantid','side','league','split','teamname','date', \
#     'gamelength','result','dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore'] ]

# lck_spring_po_drxplyaer_df = lck_spring_po_df[ (lck_spring_po_df['teamname'] == 'DRX') & (lck_spring_po_df['participantid'] <= 10 ) ]

# lck_spring_po_drxplyaer_df.head()

lck_spring_df = lck_spring[ ['gameid','playername','participantid','side','league','split','teamname','date', \
    'totalgold','gamelength','result','dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore'] ]

lck_spring_drxplyaer_df = lck_spring_df[ (lck_spring_df['teamname'] == 'DRX') & (lck_spring_df['participantid'] <= 10 ) ]

lck_spring_drxplyaer_df.head()

,gameid,playername,participantid,side,league,split,teamname,date,totalgold,gamelength,result,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
1440,ESPORTSTMNT01_2700815,Kingen,1,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,13189,2195,0,505.2301,474.5877,4763.0,6943.0,8.6105,35.0
1441,ESPORTSTMNT01_2700815,Pyosik,2,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,10792,2195,0,103.8451,640.0456,5107.0,5355.0,4.8656,58.0
1442,ESPORTSTMNT01_2700815,ZEKA,3,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,14226,2195,0,209.3303,490.6879,5589.0,7592.0,9.5945,39.0
1443,ESPORTSTMNT01_2700815,Deft,4,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,16249,2195,0,410.1048,302.7062,7171.0,5691.0,9.6219,58.0
1444,ESPORTSTMNT01_2700815,BeryL,5,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,9291,2195,0,139.2711,319.4897,4568.0,4744.0,0.9841,115.0


In [81]:
# lck_spring_po_drxplayer_score = lck_spring_po_drxplyaer_df[ ['playername','teamname','dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore'] ]
# lck_spring_po_drxplayer_score.head()

lck_spring_drxplayer_score = lck_spring_drxplyaer_df[ ['playername','teamname','dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore'] ]
lck_spring_drxplayer_score.head()

,playername,teamname,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
1440,Kingen,DRX,505.2301,474.5877,4763.0,6943.0,8.6105,35.0
1441,Pyosik,DRX,103.8451,640.0456,5107.0,5355.0,4.8656,58.0
1442,ZEKA,DRX,209.3303,490.6879,5589.0,7592.0,9.5945,39.0
1443,Deft,DRX,410.1048,302.7062,7171.0,5691.0,9.6219,58.0
1444,BeryL,DRX,139.2711,319.4897,4568.0,4744.0,0.9841,115.0


In [1]:
# X = lck_spring_po_drxplayer_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']]
A = lck_spring_drxplayer_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']]
# B = lck_spring_drxplayer_score['damagetakenperminute']
# C ]lck_spring_drxplayer_score['goldat15'
# D 'xpat15'
# E 'cspm'
# F = 'visionscore'

NameError: name 'lck_spring_drxplayer_score' is not defined

In [98]:
scaler = MinMaxScaler(feature_range=(0,5)) ## 각 칼럼 데이터 값을 0~6 범위로 변환
scaler.fit(X) ## 각 칼럼 데이터마다 변환할 함수 생
transformed_X = scaler.transform(X) ## fit에서 만들어진 함수를 실제로 데이터에 적용
transformed_X

array([[1.9559718 , 1.47733052, 1.81535492, 3.70483038, 3.46924819,
        0.63291139],
       [0.27190837, 2.26943991, 2.18421617, 2.24096608, 1.8380521 ,
        1.36075949],
       [0.71448537, 1.55440826, 2.70105083, 4.30309735, 3.89785696,
        0.75949367],
       ...,
       [0.68684203, 1.16469947, 3.29830581, 4.03300147, 4.07208816,
        0.94936709],
       [1.93686951, 0.95589562, 3.0688398 , 2.10822271, 3.50117606,
        0.85443038],
       [0.70925216, 0.49498991, 0.66802488, 0.49963127, 0.01842495,
        2.21518987]])

In [99]:
# lck_spring_po_drxplayer_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']] = transformed_X
# lck_spring_po_drxplayer_score.head()

lck_spring_drxplayer_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']] = transformed_X
lck_spring_drxplayer_score.head()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19940\3632916184.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,playername,teamname,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
1440,Kingen,DRX,1.955972,1.477331,1.815355,3.704830,3.469248,0.632911
1441,Pyosik,DRX,0.271908,2.269440,2.184216,2.240966,1.838052,1.360759
1442,ZEKA,DRX,0.714485,1.554408,2.701051,4.303097,3.897857,0.759494
1443,Deft,DRX,1.556861,0.654469,4.397384,2.550701,3.909792,1.360759
1444,BeryL,DRX,0.420543,0.734818,1.606262,1.677729,0.147356,3.164557


In [133]:
# drx_spring_po_score = lck_spring_po_drxplayer_score.groupby('playername')\
#     [['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']].mean()

# drx_spring_po_score

drx_spring_score = lck_spring_drxplayer_score.groupby('playername')\
    [['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']].mean()

drx_spring_score

,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
playername,,,,,,
BeryL,0.384423,1.152002,0.470941,0.787772,0.245233,2.657437
Clear,2.681472,2.521164,1.834656,4.209993,3.267837,0.348101
Deft,2.104303,0.992412,2.691910,2.492095,3.965646,1.041930
Jun,0.460189,0.315424,0.365644,1.120944,0.070694,1.803797
Kingen,1.948548,2.288996,2.143282,4.065381,3.272257,0.690665
Peach,0.667710,2.659788,1.322110,1.857485,3.869152,1.044304
Pyosik,0.750299,3.083310,2.032222,2.411850,2.373365,1.218354
SeTab,3.168278,1.495182,2.672100,4.821165,3.605192,0.253165
Taeyoon,1.134702,1.754415,2.733219,2.363569,3.289485,0.443038


In [141]:
# drx_spring_po_score.loc['BeryL'].index
# drx_spring_score.columns
drx_spring_score.iloc[0].values

array([0.38442297, 1.15200197, 0.47094145, 0.78777194, 0.2452326 ,
       2.65743671])

In [131]:
drx_spring_score

,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
playername,,,,,,
BeryL,0.384423,1.152002,0.470941,0.787772,0.245233,2.657437
Clear,2.681472,2.521164,1.834656,4.209993,3.267837,0.348101
Deft,2.104303,0.992412,2.691910,2.492095,3.965646,1.041930
Jun,0.460189,0.315424,0.365644,1.120944,0.070694,1.803797
Kingen,1.948548,2.288996,2.143282,4.065381,3.272257,0.690665
Peach,0.667710,2.659788,1.322110,1.857485,3.869152,1.044304
Pyosik,0.750299,3.083310,2.032222,2.411850,2.373365,1.218354
SeTab,3.168278,1.495182,2.672100,4.821165,3.605192,0.253165
Taeyoon,1.134702,1.754415,2.733219,2.363569,3.289485,0.443038


In [143]:
drx_spring_score.columns

Index(['dpm', 'damagetakenperminute', 'goldat15', 'xpat15', 'cspm',
       'visionscore'],
      dtype='object')

In [160]:
[drx_spring_score.columns] * 5

[Index(['dpm', 'damagetakenperminute', 'goldat15', 'xpat15', 'cspm',
        'visionscore'],
       dtype='object'),
 Index(['dpm', 'damagetakenperminute', 'goldat15', 'xpat15', 'cspm',
        'visionscore'],
       dtype='object'),
 Index(['dpm', 'damagetakenperminute', 'goldat15', 'xpat15', 'cspm',
        'visionscore'],
       dtype='object'),
 Index(['dpm', 'damagetakenperminute', 'goldat15', 'xpat15', 'cspm',
        'visionscore'],
       dtype='object'),
 Index(['dpm', 'damagetakenperminute', 'goldat15', 'xpat15', 'cspm',
        'visionscore'],
       dtype='object')]

In [151]:
[drx_spring_score.index[0]] * 6

['BeryL', 'BeryL', 'BeryL', 'BeryL', 'BeryL', 'BeryL']

In [163]:
drx_spring_score_df1 = pd.DataFrame( { "direction" : drx_spring_score.columns ,
"player" : [drx_spring_score.index[0]] * 6 ,
"stats" : drx_spring_score.iloc[0].values }
)

drx_spring_score_df2 = pd.DataFrame( { "direction" : drx_spring_score.columns ,
"player" : [drx_spring_score.index[2]] * 6 ,
"stats" : drx_spring_score.iloc[2].values }
)

drx_spring_score_df3 = pd.DataFrame( { "direction" : drx_spring_score.columns ,
"player" : [drx_spring_score.index[4]] * 6 ,
"stats" : drx_spring_score.iloc[4].values }
)

drx_spring_score_df4 = pd.DataFrame( { "direction" : drx_spring_score.columns ,
"player" : [drx_spring_score.index[6]] * 6 ,
"stats" : drx_spring_score.iloc[6].values }
)

drx_spring_score_df5 = pd.DataFrame( { "direction" : drx_spring_score.columns ,
"player" : [drx_spring_score.index[9]] * 6 ,
"stats" : drx_spring_score.iloc[9].values }
)


drx_spring_score_df = pd.concat( [drx_spring_score_df1,drx_spring_score_df2,drx_spring_score_df3,drx_spring_score_df4,drx_spring_score_df5]\
    ,axis=0)

drx_spring_score_df

,direction,player,stats
0,dpm,BeryL,0.384423
1,damagetakenperminute,BeryL,1.152002
2,goldat15,BeryL,0.470941
3,xpat15,BeryL,0.787772
4,cspm,BeryL,0.245233
5,visionscore,BeryL,2.657437
0,dpm,Deft,2.104303
1,damagetakenperminute,Deft,0.992412
2,goldat15,Deft,2.691910
3,xpat15,Deft,2.492095


In [173]:
import plotly.express as px

fig = px.line_polar(drx_spring_score_df , r='stats' , theta='direction', color='player', line_close=True, range_r=(0,5) ,width = 800)

fig.show()

In [172]:
import chart_studio.tools

username = 'damidaa'  # 자신의 username (plotly account를 만들어야 함)
api_key = 'OuvIp3oI14tS4Sthfgrv'    # 자신의 api key (settings > regenerate key)
  
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [174]:
chart_studio.plotly.plot(fig, filename = 'example1', auto_open=True)  # fig: 작성한 차트를 저장한 변수
  ## 위 코드를 실행하면 새로운 window로 해당 차트의 링크가 열리고, notebook에도 link를 아래에 return해줌

'https://plotly.com/~damidaa/1/'

---

### lck리그
#### split 별 분류
##### playoff 유무

In [28]:
lck = lol2022season[ lol2022season['league'] == 'LCK' ]
lck_spring = lck[ (lck['split'] == 'Spring') & (lck['playoffs']==0) ]
lck_summer = lck[ (lck['split'] == 'Summer') & (lck['playoffs']==0) ]
lck_spring_po = lck[ (lck['split'] == 'Spring') & (lck['playoffs']==1) ]
lck_summer_po = lck[ (lck['split'] == 'Summer') & (lck['playoffs']==1) ]

---

상관관계 분석

In [23]:
lck_hm_df = lck[ ['gameid','participantid','side','league','split','teamname','date', \
    'gamelength','result','totalgold','golddiffat15','xpdiffat15','towers'] ]

lck_hm_df_team = lck_hm_df[ (lck_hm_df['participantid'] == 100) | (lck_hm_df['participantid'] == 200 ) ]

lck_hm_df_team.head()

,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,xpdiffat15,towers
1450,ESPORTSTMNT01_2700815,100,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,63747,4757.0,1540.0,8.0
1451,ESPORTSTMNT01_2700815,200,Red,LCK,Spring,Liiv SANDBOX,2022-01-12 06:20:03,2195,1,67669,-4757.0,-1540.0,9.0
1462,ESPORTSTMNT01_2690695,100,Blue,LCK,Spring,DRX,2022-01-12 09:02:13,2070,0,60674,-1045.0,-735.0,3.0
1463,ESPORTSTMNT01_2690695,200,Red,LCK,Spring,Liiv SANDBOX,2022-01-12 09:02:13,2070,1,67152,1045.0,735.0,9.0
1486,ESPORTSTMNT01_2690705,100,Blue,LCK,Spring,T1,2022-01-12 10:07:10,2233,1,66455,1309.0,-170.0,7.0


In [24]:
lck_hm_df_team[ ['gamelength','result','totalgold','golddiffat15','towers'] ].groupby('result').corr().round(3)

gamelength  totalgold  golddiffat15  towers
result                                                          
0      gamelength         1.000      0.967         0.402   0.651
       totalgold          0.967      1.000         0.440   0.749
       golddiffat15       0.402      0.440         1.000   0.464
       towers             0.651      0.749         0.464   1.000
1      gamelength         1.000      0.956        -0.402  -0.023
       totalgold          0.956      1.000        -0.306   0.123
       golddiffat15      -0.402     -0.306         1.000   0.201
       towers            -0.023      0.123         0.201   1.000

In [25]:
fig = px.imshow( lck_hm_df_team[ ['gamelength','result','totalgold','golddiffat15','towers','xpdiffat15'] ].corr().round(3) , text_auto=True)

fig.show()

In [30]:
lck_spring_df = lck_spring[ ['gameid','participantid','side','league','split','teamname','date', \
    'gamelength','result','totalgold','golddiffat15','towers'] ]
lck_spring_df

,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
1440,ESPORTSTMNT01_2700815,1,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,13189,-202.0,NaN
1441,ESPORTSTMNT01_2700815,2,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,10792,240.0,NaN
1442,ESPORTSTMNT01_2700815,3,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,14226,438.0,NaN
1443,ESPORTSTMNT01_2700815,4,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,16249,2849.0,NaN
1444,ESPORTSTMNT01_2700815,5,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,9291,1432.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
49051,ESPORTSTMNT02_2672758,8,Red,LCK,Spring,Fredit BRION,2022-03-20 10:37:02,2499,0,14050,-944.0,NaN
49052,ESPORTSTMNT02_2672758,9,Red,LCK,Spring,Fredit BRION,2022-03-20 10:37:02,2499,0,18472,686.0,NaN
49053,ESPORTSTMNT02_2672758,10,Red,LCK,Spring,Fredit BRION,2022-03-20 10:37:02,2499,0,8375,-1017.0,NaN
49054,ESPORTSTMNT02_2672758,100,Blue,LCK,Spring,Kwangdong Freecs,2022-03-20 10:37:02,2499,1,71556,703.0,11.0


In [31]:
lck_spring_team_df = lck_spring_df[ (lck_spring_df['participantid'] == 100) | (lck_spring_df['participantid'] == 200 ) ]

lck_spring_team_df['result'] = np.where( lck_spring_team_df['result'] == 0 , 'lose', 'win')

lck_spring_team_df.head()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19940\3910339335.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
1450,ESPORTSTMNT01_2700815,100,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,lose,63747,4757.0,8.0
1451,ESPORTSTMNT01_2700815,200,Red,LCK,Spring,Liiv SANDBOX,2022-01-12 06:20:03,2195,win,67669,-4757.0,9.0
1462,ESPORTSTMNT01_2690695,100,Blue,LCK,Spring,DRX,2022-01-12 09:02:13,2070,lose,60674,-1045.0,3.0
1463,ESPORTSTMNT01_2690695,200,Red,LCK,Spring,Liiv SANDBOX,2022-01-12 09:02:13,2070,win,67152,1045.0,9.0
1486,ESPORTSTMNT01_2690705,100,Blue,LCK,Spring,T1,2022-01-12 10:07:10,2233,win,66455,1309.0,7.0


In [67]:
lck_spring_team_df.isna().sum() #결측값확인

gameid           0
participantid    0
side             0
league           0
split            0
teamname         0
date             0
gamelength       0
result           0
totalgold        0
golddiffat15     0
towers           0
dtype: int64

In [33]:
px.box( y= lck_spring_team_df['totalgold'], width=400, height=500 )
# 게임시간이 매우 긴 경기 totalgold 비약적으로 증가
# 따라서 15diff 지표로 확인하는것이 적절

In [67]:
lck_spring_team_df[ lck_spring_team_df['totalgold'] >= 90000 ]['teamname'] = 'outlier'

,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
10750,ESPORTSTMNT02_2552430,100,Blue,LCK,Spring,DWG KIA,2022-01-30 06:56:25,3326,win,100872,448.0,10.0
10751,ESPORTSTMNT02_2552430,200,Red,LCK,Spring,T1,2022-01-30 06:56:25,3326,lose,93420,-448.0,7.0
18922,ESPORTSTMNT02_2578644,100,Blue,LCK,Spring,T1,2022-02-12 06:05:04,2746,win,90436,-567.0,8.0


In [54]:
lck_spring_team_df['teamname'].unique()

array(['DRX', 'Liiv SANDBOX', 'T1', 'Kwangdong Freecs',
       'Nongshim RedForce', 'Hanwha Life Esports', 'KT Rolster',
       'DWG KIA', 'Gen.G', 'Fredit BRION'], dtype=object)

In [176]:
lck_spring_team_df.describe()

,participantid,gamelength,totalgold,golddiffat15,towers
count,424.000000,424.000000,424.000000,424.000000,424.000000
mean,150.000000,2057.292453,60717.971698,0.000000,6.202830
std,50.059067,378.859068,11860.644727,2590.450046,3.571809
min,100.000000,1145.000000,28982.000000,-10037.000000,0.000000
25%,100.000000,1780.000000,52468.250000,-1474.500000,3.000000
50%,150.000000,2030.500000,60665.500000,0.000000,7.000000
75%,200.000000,2316.250000,68100.000000,1474.500000,9.000000
max,200.000000,3326.000000,100872.000000,10037.000000,11.000000


> ### DRX 경기만 확인

In [183]:
lck_spring_drx_df = lck_spring_df[ (lck_spring_df['teamname'] == 'DRX') & ( (lck_spring_df['participantid'] == 100) \
    | (lck_spring_df['participantid'] == 200 ) ) ]

lck_spring_drx_df['result'] = np.where( lck_spring_drx_df['result'] == 0 , 'lose', 'win')

lck_spring_drx_df.head()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19940\2011406639.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,gameid,playername,participantid,side,league,split,teamname,date,totalgold,gamelength,result,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
1450,ESPORTSTMNT01_2700815,NaN,100,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,63747,2195,lose,1367.7813,2227.5171,27198.0,30325.0,33.6765,305.0
1462,ESPORTSTMNT01_2690695,NaN,100,Blue,LCK,Spring,DRX,2022-01-12 09:02:13,60674,2070,lose,1935.4783,2318.4928,23612.0,29371.0,34.2029,262.0
2386,ESPORTSTMNT01_2691507,NaN,100,Blue,LCK,Spring,DRX,2022-01-14 07:15:41,45102,1778,lose,1804.2520,2773.2958,22904.0,29856.0,30.3375,188.0
2423,ESPORTSTMNT01_2691515,NaN,200,Red,LCK,Spring,DRX,2022-01-14 08:42:06,40047,1556,lose,1806.4396,3931.2725,23642.0,27936.0,29.5758,162.0
4978,ESPORTSTMNT01_2704285,NaN,100,Blue,LCK,Spring,DRX,2022-01-20 06:11:55,46838,1676,lose,1674.5585,2282.4344,25652.0,30068.0,34.2959,214.0


In [184]:
lck_spring_drx_df.describe() #drx 지표통계확인

,participantid,totalgold,gamelength,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,148.780488,60768.268293,2066.317073,1881.339295,2725.212710,24603.780488,29866.365854,34.270524,274.756098
std,50.606083,12588.163567,380.109355,520.052907,490.141631,1418.306711,1147.955808,3.726990,79.211041
min,100.000000,39051.000000,1537.000000,1074.502300,1703.685900,22491.000000,27861.000000,29.575800,157.000000
25%,100.000000,53257.000000,1778.000000,1452.704700,2321.787600,23642.000000,28938.000000,31.812700,213.000000
50%,100.000000,60094.000000,1967.000000,1806.439600,2771.448100,24295.000000,29926.000000,33.676500,264.000000
75%,200.000000,69709.000000,2367.000000,2185.279000,3020.104200,25546.000000,30474.000000,34.933000,326.000000
max,200.000000,87529.000000,2887.000000,2990.254500,3948.720100,28141.000000,32598.000000,43.109000,457.000000


In [185]:
lck_spring_drx_df['totalgold'].mean() #drx 지표통계확인

60768.26829268293

In [186]:
lck_spring_team_df['teamname'] = np.where( lck_spring_team_df['totalgold'] >= 90000 , 'outlier', lck_spring_team_df['teamname'] )

lck_spring_team_df[ lck_spring_team_df['totalgold'] >= 90000 ] 

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19940\3986707623.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
10750,ESPORTSTMNT02_2552430,100,Blue,LCK,Spring,outlier,2022-01-30 06:56:25,3326,win,100872,448.0,10.0
10751,ESPORTSTMNT02_2552430,200,Red,LCK,Spring,outlier,2022-01-30 06:56:25,3326,lose,93420,-448.0,7.0
18922,ESPORTSTMNT02_2578644,100,Blue,LCK,Spring,outlier,2022-02-12 06:05:04,2746,win,90436,-567.0,8.0


In [187]:
# totalgold 평균 지표
lck_spring_team_df[ lck_spring_team_df['totalgold'] >= 90000 ]['teamname'] = 'outlier'

lck_spring_team_scored = lck_spring_team_df.groupby('teamname')[ ['towers','gamelength'] ].mean()
lck_spring_team_scored

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19940\4283900237.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,towers,gamelength
teamname,,
DRX,6.439024,2066.317073
DWG KIA,7.097561,2114.731707
Fredit BRION,6.000000,2133.750000
Gen.G,7.750000,1992.350000
Hanwha Life Esports,4.465116,2093.116279
KT Rolster,5.418605,2015.186047
Kwangdong Freecs,6.571429,2101.309524
Liiv SANDBOX,4.121951,1906.585366
Nongshim RedForce,5.466667,2094.955556


In [215]:
fig = go.Figure()
colors = ['#5F76C1',] * len(lck_spring_team_scored.index)
colors[-1] = '#C15F5F'


fig.add_trace(go.Bar(x=lck_spring_team_scored.index , y=lck_spring_team_scored['gamelength'], marker_color=colors) )
fig.update_yaxes(range=[1900,3200]) #outlier 강조하기

fig.show()

In [216]:
lck_spring_team_scored = lck_spring_team_scored.drop('outlier')

fig = go.Figure()

fig.add_trace(go.Bar(x=lck_spring_team_scored.index , y=lck_spring_team_scored['towers']) )
fig.update_yaxes(range=[3, 9])

fig.show()

In [191]:
lck_spring_team_total_gold = lck_spring_team_df.groupby('teamname')['totalgold'].mean()
lck_spring_team_total_gold

teamname
DRX                    60768.268293
DWG KIA                62660.463415
Fredit BRION           62632.363636
Gen.G                  62042.275000
Hanwha Life Esports    59222.232558
KT Rolster             59835.441860
Kwangdong Freecs       60822.285714
Liiv SANDBOX           53638.829268
Nongshim RedForce      60402.044444
T1                     62690.219512
outlier                94909.333333
Name: totalgold, dtype: float64

In [213]:
fig = go.Figure()

colors = ['#5F76C1',] * len(lck_spring_team_total_gold.index)
colors[-1] = '#C15F5F'


fig.add_trace(go.Bar(x=lck_spring_team_total_gold.index , y=lck_spring_team_total_gold.values, marker_color = colors) )
fig.update_yaxes(range=[53000, 65000])

fig.show()

In [210]:
# 15분 gold차이 평균 지표
lck_spring_team_golddiff_mean = lck_spring_team_df.groupby('teamname')['golddiffat15'].mean()
lck_spring_team_golddiff_mean = lck_spring_team_golddiff_mean.drop('outlier')
lck_spring_team_golddiff_mean

teamname
DRX                      20.097561
DWG KIA                 729.365854
Fredit BRION           -586.863636
Gen.G                   619.775000
Hanwha Life Esports    -519.813953
KT Rolster             -221.209302
Kwangdong Freecs       -221.333333
Liiv SANDBOX          -1731.609756
Nongshim RedForce        11.177778
T1                     2012.756098
Name: golddiffat15, dtype: float64

In [212]:
fig = go.Figure()
colors = ['#5F76C1',] * len(lck_spring_team_golddiff_mean.index)
colors[1] = '#C15F5F'
colors[3] = '#C15F5F'
colors[9] = '#C15F5F'

fig.add_trace( go.Bar( x=lck_spring_team_golddiff_mean.index , y=lck_spring_team_golddiff_mean.values  , marker_color = colors ) )

fig.show()

#따라서 diff 로 지표확인

In [ ]:
# 15분 gold차이 평균 지표
lck_spring_team_golddiff_mean = lck_spring_team_df.groupby('teamname')['tower'].mean()
lck_spring_team_golddiff_mean

In [ ]:
def lck_spring_team_goldmean_cal:
    for i in range


In [2]:
# 승패 확인을 위한 전처리
lol2022season.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0
